In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np

In [ ]:
### import data 
df = pd.read_csv('GISP20002019.csv')
print(df.head())

df.iloc[:, 5:len(df.columns)] = df.iloc[:, 5:len(df.columns)].apply(pd.to_numeric, errors='coerce')
## errors = coerce converts missing data to NaN
print(df.head())


In [ ]:
## narrow down from Revant's code, only look at 
drugs = ["CIP"]
drugs_name = {
    'CIP': 'Ciprofloxacin',}

drugsDict = {}
for drug in drugs:
    drugsDict[drug] = list(df.columns)

#print(drugsDict)
print(drugsDict["CIP"])


In [ ]:
#print(drugsDict["CIP"][18])
#print("CipR" not in drugsDict["CIP"][18])
#print("CLINIC" not in drugsDict["CIP"][1])

for variable in range(len(list(df.columns))):
    if "CipR" not in drugsDict["CIP"][variable]: ## so this is saying that if CipR is not in the variable (comparing susbsections of strings?)
        drugsDict["CIP"][variable] = None
   
for drug in drugs:
    drugsDict[drug] = list(filter(None, drugsDict[drug]))
    df["Total"+drug] = df[drugsDict[drug]].sum(axis=1, skipna=True)

#print(df.head())
#print(df["TotalCIP"])


In [ ]:
#now have regions
west = ['POR', 'PHX', 'HON', 'SDG', 'SFO', 'ANC', 'SEA', 'DEN', 'LVG', 'ORA', 'LBC', 'SLC', 'LAX']
southwest = ['OKC','MIN', 'ALB', 'DAL']
midwest = ['KCY','CHI', 'PON', 'CIN', 'JAC', 'IND', 'STL','DTR', 'MIL', 'COL', 'CLE']
southeast = ['GRB', 'NOR','WDC','MIA', 'BHM','FBG','ATL', 'RIC']
northeast = ['BUF','BOS', 'CAM', 'NYC', 'BAL', 'PHI']

df['Region'] = df['CLINIC'].apply(lambda x: 
    'West' if (x in west) else (
        'Southwest' if (x in southwest) else(
            'Midwest' if (x in midwest) else(
                'Southeast' if (x in southeast) else(
                    'Northeast' if (x in northeast) else 'Other'))))) 

#now by location 
west = ['POR', 'PHX', 'HON', 'SDG', 'SFO', 'ANC', 'SEA', 'DEN', 'LVG', 'ORA', 'LBC', 'SLC', 'LAX']
southwest = ['OKC','MIN', 'ALB', 'DAL']
midwest = ['KCY','CHI', 'PON', 'CIN', 'JAC', 'IND', 'STL','DTR', 'MIL', 'COL', 'CLE']
southeast = ['GRB', 'NOR','WDC','MIA', 'BHM','FBG','ATL', 'RIC']
northeast = ['BUF','BOS', 'CAM', 'NYC', 'BAL', 'PHI']

df['Region'] = df['CLINIC'].apply(lambda x: 
    'West' if (x in west) else (
        'Southwest' if (x in southwest) else(
            'Midwest' if (x in midwest) else(
                'Southeast' if (x in southeast) else(
                    'Northeast' if (x in northeast) else 'Other'))))) 
    

In [ ]:
### nb - only have one drug (CIP) BUT it is editable

overall = pd.DataFrame([],[])
for year in list(set(df["YEAR"])): #so gets all possible years from YEAR dataframe
     df2 = df[df["YEAR"]==year]     
     new = {'Year': year} # so a new dictionary with year?
    
     new['OverallTotal'] = df2["TOTAL"].sum() #so adding a new entry into new with the sum of total... Total being the number of tests carried out FOR THAT YEAR I think
    
     pass
     for drug in drugs:
           new[drug+'_OverallTotal'] = df2["Total"+drug].sum() #
           new[drug+'_OverallPct'] = (new[drug+'_OverallTotal']/new['OverallTotal'] )*100 #percentage of overall cases that are resistant to a drug in that year 
           
           for gender in list(set(df["GENDERSP"])):       
                df3 = df2[df2["GENDERSP"]==gender]
            
                new[gender+"_Total"] = df3["TOTAL"].sum()
                new[gender+"_"+drug+"Total"] = df3["Total"+drug].sum()
                new[gender+"_"+drug+"Pct"] = (new[gender+"_"+drug+"Total"] / new[gender+"_Total"])*100
        
           for region in list(set(df["Region"])):       
                df4 = df2[df2["Region"]==region]
            
                new[region+"_Total"] = df4["TOTAL"].sum()
                new[region+"_"+drug+"_Total"] = df4["Total"+drug].sum()
                new[region+"_"+drug+"_Pct"] = (new[region+"_"+drug+"_Total"]/new[region+"_Total"])*100
    
     overall = overall.append(new, ignore_index = True)
overall["Year"] = overall["Year"].astype(int)

In [ ]:
#try and get only CIP data from overall (which is where the the overall data is)

#CIPR_data = df.loc[df['TotalCIP'] >= 0] ##from original data frame


In [ ]:
### now... how do I dissagregate data? 
print(overall)



